# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,10.07,77,7,2.02,AU,1693553887
1,1,atafu village,-8.5421,-172.5159,28.39,79,11,6.88,TK,1693553888
2,2,bethel,41.3712,-73.4140,15.36,80,0,0.00,US,1693553754
3,3,buka,40.8108,69.1986,30.47,13,0,3.64,UZ,1693553888
4,4,puerto ayora,-0.7393,-90.3518,24.19,91,80,2.24,EC,1693553888


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "humidity",
    color = "city",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600, 
    scale = 1
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0) 
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,sesheke,-17.4759,24.2968,26.84,16,0,3.74,ZM,1693553900
57,57,saint-pierre,-21.3393,55.4781,24.82,61,0,3.09,RE,1693553902
124,124,gangneung,37.7556,128.8961,26.31,68,0,2.36,KR,1693553917
126,126,constantia,44.1833,28.6500,26.50,47,0,3.16,RO,1693553918
131,131,cavezzo,44.8364,11.0286,24.38,63,0,1.29,IT,1693553919
165,165,beira,-19.8436,34.8389,24.05,69,0,4.12,MZ,1693553926
200,200,kailua-kona,19.6406,-155.9956,26.98,79,0,4.12,US,1693553853
222,222,sayat,38.7839,63.8803,26.93,12,0,3.64,TM,1693553940
248,248,xilin hot,43.9667,116.0333,25.49,19,0,2.50,CN,1693553946
249,249,soto la marina,23.7667,-98.2167,24.69,72,0,1.16,MX,1693553946


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity" ]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,sesheke,ZM,-17.4759,24.2968,16,
57,saint-pierre,RE,-21.3393,55.4781,61,
124,gangneung,KR,37.7556,128.8961,68,
126,constantia,RO,44.1833,28.6500,47,
131,cavezzo,IT,44.8364,11.0286,63,
165,beira,MZ,-19.8436,34.8389,69,
200,kailua-kona,US,19.6406,-155.9956,79,
222,sayat,TM,38.7839,63.8803,12,
248,xilin hot,CN,43.9667,116.0333,19,
249,soto la marina,MX,23.7667,-98.2167,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 12,000
params = {
    "categories":"accommodation.hotel",
    "apikey": geoapify_key,
    "limit":25
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longtiude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longtiude},{latitude},{radius}"
    params["bias"] = f"circle:{longtiude},{latitude},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sesheke - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
gangneung - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
cavezzo - nearest hotel: No hotel found
beira - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
sayat - nearest hotel: No hotel found
xilin hot - nearest hotel: No hotel found
soto la marina - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
big spring - nearest hotel: No hotel found
maraba - nearest hotel: No hotel found
caboolture - nearest hotel: No hotel found
boa vista - nearest hotel: No hotel found
maralal - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
ondjiva - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
49,sesheke,ZM,-17.4759,24.2968,16,No hotel found
57,saint-pierre,RE,-21.3393,55.4781,61,No hotel found
124,gangneung,KR,37.7556,128.8961,68,No hotel found
126,constantia,RO,44.1833,28.6500,47,No hotel found
131,cavezzo,IT,44.8364,11.0286,63,No hotel found
165,beira,MZ,-19.8436,34.8389,69,No hotel found
200,kailua-kona,US,19.6406,-155.9956,79,No hotel found
222,sayat,TM,38.7839,63.8803,12,No hotel found
248,xilin hot,CN,43.9667,116.0333,19,No hotel found
249,soto la marina,MX,23.7667,-98.2167,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(    
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "city",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600, 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)